In [1]:
import numpy as np
import pandas as pd

In [2]:
temp_df = pd.DataFrame({
    'key1': [1,1,1,2,2,3],
    'key2': [12,np.nan, 45, 12, 45, 12],
    'data': ['Dynamo', 'Snax', 'Mortal', 'Scout', 'Joker', 'Krutika']
})

groupby_obj = temp_df.groupby(['key1'])
key_2_gp = groupby_obj['key2']

### Aggregation - df.agg()

In [5]:
key_2_gp.agg('mean')

key1
1    28.5
2    28.5
3    12.0
Name: key2, dtype: float64

In [6]:
key_2_gp.agg(lambda x: x.max() - x.min())

key1
1    33.0
2    33.0
3     0.0
Name: key2, dtype: float64

In [7]:
key_2_gp.agg(['mean', 'std', 'min', 'max'])

,mean,std,min,max
key1,,,,
1,28.5,23.334524,12.0,45.0
2,28.5,23.334524,12.0,45.0
3,12.0,NaN,12.0,12.0


In [8]:
key_2_gp.agg([('agg1','mean'), ('agg2', 'std')])

,agg1,agg2
key1,,
1,28.5,23.334524
2,28.5,23.334524
3,12.0,NaN


In [38]:
groupby_obj.agg({
    'key2': ['mean', 'std', 'min', 'max'],
    'data': len
})

key2                        data
      mean        std   min   max  len
key1                                  
1     28.5  23.334524  12.0  45.0    3
2     28.5  23.334524  12.0  45.0    2
3     12.0        NaN  12.0  12.0    1

In [36]:
df = pd.DataFrame({
    'data1': [1,2,3,4,5],
    'data2': [1,2,3,4,5],
    'data3': [1,2,3,4,5]
}, index = ['a', 'b', 'c', 'd', 'e'])

df.unstack()

data1  a    1
       b    2
       c    3
       d    4
       e    5
data2  a    1
       b    2
       c    3
       d    4
       e    5
data3  a    1
       b    2
       c    3
       d    4
       e    5
dtype: int64

### Split Apply Combine - apply()

In [41]:
import seaborn as sns
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [46]:
def tempfun(df, n = 5, column = 'col_name'):
    return df.sort_values(by=column)[-n:]

df.groupby(['smoker', 'day'], observed = False)[['total_bill', 'tip']].apply(tempfun, n = 1, column = 'total_bill')

total_bill    tip
smoker day                        
Yes    Thur 197       43.11   5.00
       Fri  95        40.17   4.73
       Sat  170       50.81  10.00
       Sun  182       45.35   3.50
No     Thur 142       41.19   5.00
       Fri  94        22.75   3.25
       Sat  212       48.33   9.00
       Sun  156       48.17   5.00

### Pivot Tables - pivot_table()

In [56]:
df.pivot_table(['total_bill'], index = ['day', 'smoker'], columns = ['time'], aggfunc = len)

C:\Users\shrir\AppData\Local\Temp\ipykernel_1212\1273154217.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  df.pivot_table(['total_bill'], index = ['day', 'smoker'], columns = ['time'], aggfunc = len)


total_bill       
time             Lunch Dinner
day  smoker                  
Thur Yes          17.0    NaN
     No           44.0    1.0
Fri  Yes           6.0    9.0
     No            1.0    3.0
Sat  Yes           NaN   42.0
     No            NaN   45.0
Sun  Yes           NaN   19.0
     No            NaN   57.0

### Cross Tabulation - crosstab()

In [58]:
pd.crosstab([df['smoker'], df['time']], [df['day'], df['sex']])

day           Thur         Fri         Sat         Sun       
sex           Male Female Male Female Male Female Male Female
smoker time                                                  
Yes    Lunch    10      7    3      3    0      0    0      0
       Dinner    0      0    5      4   27     15   15      4
No     Lunch    20     24    0      1    0      0    0      0
       Dinner    0      1    2      1   32     13   43     14